# IMPORT

In [22]:
import pandas as pd
import numpy as np
import re

# FUNCTION

In [18]:
def clean_text(
        text,
        methods=['rmv_link', 'rmv_punc', 'lower', 'rmv_space'],
        custom_punctuation = '!"#$%&\'()*+,.-:;<=>?@[\\]^_/`{|}~”“',
    ):
    cleaned_text = text
    for method in methods:
        if method == 'rmv_link':
            # Remove link
            cleaned_text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', cleaned_text)
            cleaned_text = "".join(cleaned_text)
        elif method == 'rmv_punc':
            # Remove punctuation
            cleaned_text = re.sub('[%s]' % re.escape(custom_punctuation), '' , cleaned_text)
        elif method == 'lower':
            # Lowercase
            cleaned_text = cleaned_text.lower()
        elif method == 'rmv_space':
            # Remove extra space
            cleaned_text = re.sub(' +', ' ', cleaned_text)
            cleaned_text = cleaned_text.strip()
    return cleaned_text

In [30]:
def preprocessing(text):
    text = text.replace("'s", "")
    text = re.sub(r'[^a-zA-Z0-9\s\.,!?\-\%\$€£]', ' ', text)
    text = clean_text(
        text=text,
        custom_punctuation="#$}{!)(?|-#$%&<=>?@[\\]^_/`{|}~”“"
    )
    return text

# LOAD DATA

In [15]:
data_path = r"C:\APAC\all_projects\finetuning-airflow-project\project_2_training\data\all-data.csv"
df = pd.read_csv(data_path, encoding="ISO-8859-1", header=None)
df.columns = ["label", "title"]

# PREPROCESSING

## *Sentence Processing*

In [31]:
df["title"].apply(preprocessing)

0       according to gran , the company has no plans t...
1       technopolis plans to develop in stages an area...
2       the international electronic industry company ...
3       with the new production plant the company woul...
4       according to the company updated strategy for ...
                              ...                        
4841    london marketwatch share prices ended lower in...
4842    rinkuskiai beer sales fell by 6.5 per cent to ...
4843    operating profit fell to eur 35.4 mn from eur ...
4844    net sales of the paper segment decreased to eu...
4845    sales in finland decreased by 10.5 in january ...
Name: title, Length: 4846, dtype: object

## *Label Processing*